In [1]:
import os
import sys

sys.path.append("../../")
# os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

import tensorflow as tf
import tensorflow_ranking as tfr
import tensorflow_datasets as tfds
from tensorflow.data.experimental import AUTOTUNE
tf.random.set_seed(214)

from lmtk.feature_extraction import FeatureExtractor, FEATURE_BIT, FEATURE_MASK
from lmtk.tf_model.layers.embeddings import SparseEmbeddingLayer
import tfutils

fe = FeatureExtractor('../../config/dynamic_pricing/0.0.1.conf')

schema = dict()

for k,v in fe.feature_conf_map.items():
    schema[f'{k}'] = tf.io.FixedLenFeature(shape = (int(v.args[0], )) ,dtype = tf.int64)
    
schema['trigger_type'] = tf.io.FixedLenFeature(shape = (1,), dtype = tf.string)
schema['bundle_price'] = tf.io.FixedLenFeature(shape = (1,),dtype = tf.int64)
schema['if_buy_pop_gift'] = tf.io.FixedLenFeature(shape = (1,),dtype = tf.int64)


def decode_fn(msg):
    data = tf.io.parse_single_example(msg, schema)
    y = data.pop('if_buy_pop_gift')
    for k in fe.feature_conf_map:
        data[k] = tf.map_fn(lambda x: x & FEATURE_MASK, data[k])
    # y = tf.cast(y, tf.float32)
    return data, y

batchsize = 128

easyDataPositive = tf.data.TFRecordDataset('../../data/tfrecord/0.0.1.train.pos.tfrecord', num_parallel_reads=AUTOTUNE).map(decode_fn)
easyDataNegative = tf.data.TFRecordDataset('../../data/tfrecord/0.0.1.train.neg.tfrecord', num_parallel_reads=AUTOTUNE).map(decode_fn)
hard1DataPositive = tf.data.TFRecordDataset('../../data/tfrecord/0.0.1.train.pos.hard1.tfrecord', num_parallel_reads=AUTOTUNE).map(decode_fn)
hard1DataNegative = tf.data.TFRecordDataset('../../data/tfrecord/0.0.1.train.neg.hard1.tfrecord', num_parallel_reads=AUTOTUNE).map(decode_fn)
hard2DataPositive = tf.data.TFRecordDataset('../../data/tfrecord/0.0.1.train.pos.hard2.tfrecord', num_parallel_reads=AUTOTUNE).map(decode_fn)
hard2DataNegative = tf.data.TFRecordDataset('../../data/tfrecord/0.0.1.train.neg.hard2.tfrecord', num_parallel_reads=AUTOTUNE).map(decode_fn)

traindata = (
    tf.data.Dataset.sample_from_datasets([
        tf.data.Dataset.zip((easyDataPositive, easyDataNegative.shuffle(256 * 4, reshuffle_each_iteration = True))),
        tf.data.Dataset.zip((hard1DataPositive, hard1DataNegative)),
        tf.data.Dataset.zip((hard2DataPositive, hard2DataNegative)),
    ], weights=[1.0 /3, 1.0 / 3, 1.0 / 3])
    .batch(batchsize)
    # .cache()
    .prefetch(AUTOTUNE)
) 


traindata_plain = (
    tf.data.TFRecordDataset([
        '../../data/tfrecord/0.0.1.train.pos.tfrecord',
        '../../data/tfrecord/0.0.1.train.neg.tfrecord'
    ], num_parallel_reads=AUTOTUNE)
    .map(decode_fn)
    .shuffle(batchsize * 4, reshuffle_each_iteration=True)
    .batch(batchsize)
    # .shuffle(batchsize  * 4 )
    # .cache()
    .prefetch(AUTOTUNE)
) 

validdata = (
    tf.data.TFRecordDataset('../../data/tfrecord/0.0.1.valid.tfrecord', num_parallel_reads=AUTOTUNE)
    .map(decode_fn)
    .batch(batchsize)
    # .cache()
    .prefetch(AUTOTUNE)
)

d:\Anaconda\envs\tf210\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from itertools import groupby
from collections import defaultdict

with open('../../data/vocab/0.0.1.txt', 'r') as f:
    vocab = [int(v) for v in f.readlines()] 

vocab_slot = defaultdict(set)
slot_name_map = {
    f.slot : name
    for name, f in fe.feature_conf_map.items()
}

for k, g in groupby(vocab, lambda x: x>> FEATURE_BIT):
    name = slot_name_map[k]
    vocab_slot[name].update([FEATURE_MASK & v for v in list(g)])

tf.keras.backend.set_floatx('float32')

embedding_size : int = 16

inputs, embeddings, inputs_context, embedding_context = dict(), dict(), dict(), dict()

for idx, (name, conf) in enumerate(fe.feature_conf_map.items()):
    this_slot_vocab = list(vocab_slot.get(name))
    conf = fe.feature_conf_map.get(name)
    inputs[name], embeddings[name] = tfutils.inputs.uint64_to_embedding(
        shape = (int(conf.args[0]),), vocab = this_slot_vocab, embedding_size=embedding_size, name = name, mask = FEATURE_MASK
    )

name = 'trigger_type'
vocab = "account_lvl_trigger,mainbase_lvl_trigger,campaign_stage_trigger,tech_unlock_trigger,building_trigger-10001,building_trigger-10002,building_trigger-10003,building_trigger-10004,building_trigger-10005,building_trigger-10006,building_trigger-10014,building_trigger-10015,building_trigger-10018,building_trigger-10010,building_trigger-10011,building_trigger-10012,building_trigger-10013,building_trigger-10024,building_trigger-10027,building_trigger-10028,building_trigger-10029,building_trigger-10030,card_unlock_trigger_ground_force,card_unlock_trigger_air_force,gift_login,gvg_trigger,teleport_trigger,population_consume_trigger,blueprint_max_trigger_ground_force,blueprint_max_trigger_air_force,advancecard_unlock_trigger_ground_force,advancecard_unlock_trigger_air_force,mechanician_trigger,tech_lv_trigger,multi_tech_lv_trigger,blackmall_trigger,all_officer_lv_trigger,officer_lv_trigger,drawing_trigger,replacement_trigger,replacement_recycle_trigger,card_unlock_trigger,building_trigger-10020"
vocab = vocab.split(',')
bundle_trigger_type_embedding_layer = tf.keras.Sequential([
    tf.keras.layers.StringLookup(vocabulary=vocab, oov_token=0),
    tf.keras.layers.Embedding(input_dim = len(vocab) + 1, output_dim = embedding_size, mask_zero = True)
])
inputs[name] = tf.keras.Input(shape =(1,), name = name, dtype = tf.string)
embeddings[name] = bundle_trigger_type_embedding_layer(inputs[name])

name = 'bundle_price'
vocab = [99, 499, 999, 1499, 1999, 2999, 4999, 9999]
bundle_price_embedding_layer = tf.keras.Sequential([
    tf.keras.layers.IntegerLookup(vocabulary=vocab, oov_token=0),
    tf.keras.layers.Embedding(input_dim = len(vocab) + 1, output_dim = embedding_size, mask_zero = True)
])
inputs[name] = tf.keras.Input(shape =(1,), name = name, dtype = tf.int64)
embeddings[name] = bundle_price_embedding_layer(inputs[name])

# name = 'gift_contents'
# vocab = 'airforce_equipment_exp,gold_50000_card,officer_exp_card_2,mothersdayframe,rush_common_3h,armor_406,fragment_star05,officer_skill_power_03_random_chest,metal_500000_card,rush_tech_common_3m,l_coin_paid,oil_50000_card,valentine_chocolates,easter_egg,mothersday_chocolates,valentine_kiss,maintenance_props_504,rush_build_common_60m,rush_tech_common_8h,chest_select_resource_50k,rush_common_8h,journey_slotmachine_coin,action_point_middle,220303,officer_skill_order_02_random_chest,officer_skill_item_power_01,rush_common_3m,journeyframe,christmas_snowman,officer_skill_order_01_random_chest,christmas_pancake,vip_point_10,fatherframe,airforce_ammunition,top_key,officer_skill_power_02_random_chest,ammunition,metal_5000_card,oil_500000_card,father_egg,heavy_cannon_203,410601,random_equipment_power_star05_chest,elements,autumnframe,chest_select_replacement_silver,officer_exp_card_3,officer_exp_card_1,rush_common_60m,easterframe,gold_500000_card,vip_point_100,metal_50000_card,chest_select_replacement_golden,christmasframe,rush_common_24h,summer_beer,chest_select_resource_500k,rush_tech_common_60m,action_point_large,board,armor_404,common_equipment_exp,rush_common_1m,officer_skill_power_04_random_chest,l_coin,rush_build_common_3m,officer_skill_item_common,camp_key,officer_skill_freedom_03_random_chest,chest_select_resource_150k,cement,vip_point_1000,summerframe,maintenance_props_503,superheavy_suspension_203,heavy_cannon_205,oil_5000_card,rush_build_common_8h,mothersday_kiss,valentineframe,brick,220401,battlepass_metal,gold_5000_card,officer_skill_common_random_chest,autumn_apple,superheavy_suspension_204,officer_skill_freedom_01_random_chest'
# vocab = vocab.split(',')
# bundle_gift_contents_embedding_layer = tf.keras.Sequential([
#     tf.keras.layers.StringLookup(vocabulary=vocab, oov_token=0),
#     SparseEmbeddingLayer(input_dim = len(vocab) + 1, output_dim = embedding_size, )
# ])
# inputs_context[name] = tf.keras.Input(shape =(None,), name = name, dtype = tf.string, sparse = True)
# embedding_context[name] = bundle_gift_contents_embedding_layer(inputs_context[name])

d:\Anaconda\envs\tf210\lib\site-packages\numpy\core\numeric.py:2463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return bool(asarray(a1 == a2).all())


In [3]:

# cross_target = [
#     ("lifespan", "lvl"),
#     ("lifespan", "region"),
#     ("lifespan", "sid"),
#     ("lifespan", "lped"),
#     ("lifespan", "pcnt"),
#     ("lifespan", "tot_p"),
#     ("lifespan", "avg_pp"),
#     ("lifespan", "max_pp"),
#     ("lifespan", "max_5rpp"),
#     ("max_5rpp", "avg_5rpp"),
#     ("max_5rpp", "max_pp"),
#     ("max_5rpp", "region"),
#     ("max_5rpp", "lvl"),
#     ("lifespan", "p3d"),
#     ("lifespan", "p7d"),
# ]

# for slot1, slot2 in cross_target:
#     num_bins = len(vocab_slot.get(slot1)) * len(vocab_slot.get(slot2))
#     cross_inputs = (inputs.get(slot1), inputs.get(slot2))
#     embeddings[f'x_{slot1}_{slot2}'] = tfutils.inputs.cross_to_embedding(cross_inputs, num_bins= min(num_bins, 64), embedding_size=embedding_size, names = [slot1, slot2])

# for slot1 in ["lifespan", "sid", "max_5rpp", "avg_5rpp", 'pcnt', "tot_p", "max_pp","avg_pp","p7d"]:
#     num_bins = len(vocab_slot.get(slot1)) * len([99, 499, 999, 1499, 1999, 2999, 4999, 9999])
#     cross_inputs = (inputs.get(slot1), inputs.get('bundle_price'))
#     embeddings[f'x_{slot1}_price'] = tfutils.inputs.cross_to_embedding(cross_inputs, num_bins= min(num_bins, 64), embedding_size=embedding_size, names = [slot1, 'price'])

# for slot1 in ["lifespan", "max_5rpp", "avg_5rpp", 'pcnt', "tot_p", "avg_pp","p7d"]:
# # for slot1 in fe.feature_conf_map.keys():
#     num_bins = len(vocab_slot.get(slot1)) * len([99, 499, 999, 1499, 1999, 2999, 4999, 9999]) * 43
#     cross_inputs = (inputs.get(slot1), inputs.get('bundle_price'), inputs.get('trigger_type'))
#     embeddings[f'x_{slot1}_trigger_price'] = tfutils.inputs.cross_to_embedding(cross_inputs, num_bins= min(num_bins, 1024), embedding_size=embedding_size, names = [slot1, 'trigger_price'])


In [4]:

class GatedLinearUnit(tf.keras.layers.Layer):
    def __init__(self, units):
        super().__init__()
        self.linear = tf.keras.layers.Dense(units)
        self.sigmoid = tf.keras.layers.Dense(units, activation="sigmoid")

    def call(self, inputs):
        return self.linear(inputs) * self.sigmoid(inputs)

class GatedResidualNetwork(tf.keras.layers.Layer):
    def __init__(self, units, dropout_rate):
        super().__init__()
        self.units = units
        self.elu_dense = tf.keras.layers.Dense(units, activation="elu")
        self.linear_dense = tf.keras.layers.Dense(units)
        self.dropout = tf.keras.layers.Dropout(dropout_rate)
        self.gated_linear_unit = GatedLinearUnit(units)
        self.layer_norm = tf.keras.layers.BatchNormalization()
        self.project = tf.keras.layers.Dense(units)

    def call(self, inputs):
        x = self.elu_dense(inputs)
        x = self.linear_dense(x)
        x = self.dropout(x)
        if inputs.shape[-1] != self.units:
            inputs = self.project(inputs)
        x = inputs + self.gated_linear_unit(x)
        x = self.layer_norm(x)
        return x

class VariableSelection(tf.keras.layers.Layer):
    def __init__(self, num_features, units, dropout_rate):
        super().__init__()
        self.grns = list()
        # Create a GRN for each feature independently
        for idx in range(num_features):
            grn = GatedResidualNetwork(units, dropout_rate)
            self.grns.append(grn)
        # Create a GRN for the concatenation of all the features
        self.grn_concat = GatedResidualNetwork(units, dropout_rate)
        self.softmax = tf.keras.layers.Dense(units=num_features, activation="softmax")

    def call(self, inputs):
        v = tf.keras.layers.concatenate(inputs)
        v = self.grn_concat(v)
        v = tf.expand_dims(self.softmax(v), axis=-1)

        x = []
        for idx, input in enumerate(inputs):
            x.append(self.grns[idx](input))
        x = tf.stack(x, axis=1)

        outputs = tf.squeeze(tf.matmul(v, x, transpose_a=True), axis=1)
        return outputs

class FieldVariableSelection(tf.keras.layers.Layer):
    def __init__(self, num_features, units, dropout_rate):
        super().__init__()
        self.grn = GatedResidualNetwork(units, dropout_rate)
        self.grn_concat = GatedResidualNetwork(units, dropout_rate)
        self.softmax = tf.keras.layers.Dense(units=num_features, activation="softmax")

    def call(self, inputs):
        v = tf.keras.layers.Flatten()(inputs)
        v = self.grn_concat(v)
        v = tf.expand_dims(self.softmax(v), axis=-1)
        x = self.grn(inputs)
        outputs = tf.squeeze(tf.matmul(v, x, transpose_a=True), axis=1)
        return outputs
    
class VariableSelection(tf.keras.layers.Layer):
    def __init__(self, num_features, units, dropout_rate):
        super().__init__()
        self.grns = list()
        # Create a GRN for each feature independently
        for idx in range(num_features):
            grn = GatedResidualNetwork(units, dropout_rate)
            self.grns.append(grn)
        # Create a GRN for the concatenation of all the features
        self.grn_concat = GatedResidualNetwork(units, dropout_rate)
        self.softmax = tf.keras.layers.Dense(units=num_features, activation="softmax")

    def call(self, inputs):
        v = tf.keras.layers.concatenate(inputs)
        v = self.grn_concat(v)
        v = tf.expand_dims(self.softmax(v), axis=-1)

        x = []
        for idx, input in enumerate(inputs):
            x.append(self.grns[idx](input))
        x = tf.stack(x, axis=1)

        outputs = tf.squeeze(tf.matmul(v, x, transpose_a=True), axis=1)
        return outputs

class PairwiseRankingModelWrapper(tf.keras.Model):
    def __init__(self, model : tf.keras.Model):
        super().__init__()
        self.model = model

    def call(self, x, training = True):
        return self.model(x, training)

    def train_step(self, data):
        (pos, _), (neg, _) = data
        with tf.GradientTape() as tape:
            ypos = self(pos, training = True)
            yneg = self(neg, training = True)

            loss = self.compiled_loss(ypos, yneg)

        gradients = tape.gradient(loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))
        return {m.name: m.result() for m in self.metrics}
    
    def test_step(self, data):
        data, y_true = data
        y_pred = self.model(data)
        self.compiled_metrics.update_state(y_true, y_pred)
        return {m.name: m.result() for m in self.metrics}

In [5]:
size_splits = {name : x.shape[1] for name, x in  embeddings.items()}

outputs = tf.concat(embeddings.values(), axis = 1)

residual = [tf.math.reduce_max(e, axis = 1) if e.shape[1] > 1 else tf.squeeze(e, 1) for e in embeddings.values()]
residual = tf.concat(residual, axis = 1)
residual = tf.keras.layers.Flatten()(residual)
residual = tf.keras.layers.Dense(32, activation = 'leaky_relu', kernel_regularizer = tf.keras.regularizers.L2(0.01))(residual)
# residual = VariableSelection(len(residual), 16, 0.1)(residual)

# residual = [FieldVariableSelection(e.shape[1], 32, 0.1)(e) for e in embeddings.values()]
# residual = tf.concat(residual, axis = 1)

outputs = tfutils.layers.attn.BatchNormTransformerEncoder(16, 4, 16)(outputs)
outputs = tf.keras.layers.Dropout(0.1)(outputs)
outputs = tfutils.layers.attn.BatchNormTransformerEncoder(16, 4, 16)(outputs)

outputs = tf.split(outputs, size_splits.values(), axis= 1)
outputs = [tf.math.reduce_mean(x, axis = 1) for x in outputs] + [tf.math.reduce_max(x, axis = 1) for x in outputs]
# outputs = [tf.math.reduce_max(x, axis = 1) for x in outputs]

outputs = tf.concat(outputs, axis = 1)
outputs = tf.keras.layers.Flatten()(outputs)
outputs = tf.keras.layers.Dense(64, activation = 'leaky_relu', kernel_regularizer = tf.keras.regularizers.L2(0.01))(outputs)

outputs = tf.concat([outputs,residual], axis = 1)

outputs = tf.keras.layers.Dense(1)(outputs)
outputs = tf.math.sigmoid(outputs)

ranking_model = tf.keras.Model(inputs = list(inputs.values()), outputs = outputs)
sum([tf.keras.backend.count_params(p) for p in ranking_model.trainable_variables])

2562449

In [6]:
ranking_model = tf.keras.Model(inputs = list(inputs.values()), outputs = outputs)
lr = tf.keras.optimizers.schedules.CosineDecay(1e-4,decay_steps=200,alpha = 0.05)
ranking_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
    loss = tf.keras.losses.Hinge(),
    # loss = tfr.keras.losses.PairwiseHingeLoss(),
    metrics=[
        tf.keras.metrics.BinaryAccuracy(threshold=0.5),
        tf.keras.metrics.AUC(name = 'auc'), 
        # tfr.keras.metrics.NDCGMetric(name = 'ndgc'),
    ],
)
callbacks = [
    # tf.keras.callbacks.ModelCheckpoint("../../savedmodel/checkpoint/0.0.1/0", monitor=f'val_auc', save_best_only=True, mode="max"),
    tf.keras.callbacks.EarlyStopping(monitor='val_auc',min_delta=0.001,patience=3,mode='max'),
]

# ranking_model.evaluate(validdata.take(300))

# ranking_model.fit(traindata_plain.shuffle(batchsize * 4).repeat(100), validation_data=validdata.shuffle(batchsize * 4).repeat(100), epochs= 30, steps_per_epoch = 1000,validation_steps=500,callbacks=callbacks)
# ranking_model.fit(traindata_plain.take(500), validation_data=validdata.take(300), epochs= 30, callbacks=callbacks)
ranking_model.fit(traindata_plain, validation_data=validdata, epochs= 30, callbacks=callbacks)
# ranking_model.save_weights('../../savedmodel/dynamic_pricing/0.0.1/init_weights/')

300/300 [==============================] - 42s 129ms/step - loss: 3.2882 - binary_accuracy: 0.0740 - auc: 0.5138
Epoch 1/30
5729/5729 [==============================] - 1028s 178ms/step - loss: 1.9163 - binary_accuracy: 0.9546 - auc: 0.9792 - val_loss: 1.5267 - val_binary_accuracy: 0.9401 - val_auc: 0.5860
Epoch 2/30
5729/5729 [==============================] - 1005s 175ms/step - loss: 1.2673 - binary_accuracy: 0.9537 - auc: 0.9500 - val_loss: 1.1582 - val_binary_accuracy: 0.9401 - val_auc: 0.5814
Epoch 3/30
5729/5729 [==============================] - ETA: 0s - loss: 1.0369 - binary_accuracy: 0.9597 - auc: 0.9585

In [ ]:
def loss_fn(ypos, yneg):
    loss = tf.math.maximum(0.0 , 2e-1 + yneg - ypos)
    return tf.reduce_sum(loss)

model = PairwiseRankingModelWrapper(ranking_model)

lr = tf.keras.optimizers.schedules.CosineDecay(1e-4,decay_steps=200,alpha = 0.05)

model.compile(
    # optimizer=tf.keras.optimizers.SGD(learning_rate=lr, momentum=1e-3, nesterov=True),
    optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
    loss = loss_fn,
    metrics=[tf.keras.metrics.AUC(name = 'auc')],
)

callbacks = [
    tf.keras.callbacks.ModelCheckpoint("../../savedmodel/checkpoint/0.0.1/1", monitor=f'val_auc', save_best_only=True, mode="max"),
    tf.keras.callbacks.EarlyStopping(monitor='val_auc',min_delta=5e-3,patience=3,mode='max'),
]

model.fit(traindata, validation_data=validdata, epochs= 1, callbacks=callbacks)

In [ ]:

target_price = [99, 499, 999, 1499, 1999, 2999, 4999, 9999]
target_price_tensor = tf.convert_to_tensor(target_price)
i2p = dict(zip(range(len(target_price)), target_price))


def get_pred(x):
    shape = x.get('bundle_price').shape[0]

    # def repeat(x):
    #     v = tf.expand_dims(x, 2)
    #     v = tf.repeat(v, len(target_price), axis = 2)
    #     v = tf.split(v, len(target_price), axis = 2)
    #     v = tf.concat(v, axis = 0)
    #     v = tf.squeeze(v)
    #     return v

    def repeat(x):
        x = tf.repeat(tf.expand_dims(x, 0), len(target_price), axis = 0)
        x = tf.reshape(x, (shape * len(target_price), -1))
        return x


    jitted_repeat = tf.function(repeat)

    for k in x.keys():
        if k != 'bundle_price':
            if k != 'trigger_type':
                x[k] = jitted_repeat(x[k])
            else:
                x[k] = repeat(x[k])
        else:
            x['bundle_price'] = tf.reshape(tf.repeat(target_price_tensor, shape), (-1,1))

    ypred = model(x)
    # ypred = tf.reshape(ypred, (len(target_price), shape))
    # ypred = tf.transpose(ypred)
    ypred = tf.reshape(model(x), (shape, -1))

    return ypred

In [ ]:
 # 计算badcase
from tqdm import tqdm
if_buy_pop_gifts = []
ypreds = []
bundle_prices = []

for x, if_buy_pop_gift in tqdm(validdata.unbatch().batch(16)):
    if_buy_pop_gifts.append(if_buy_pop_gift)
    bundle_prices.append(x.get('bundle_price'))
    ypred = get_pred(x)
    ypred = tf.argmax(ypred, axis = 1)
    ypred = tf.matmul( tf.one_hot(ypred, len(target_price)) ,  tf.reshape(tf.convert_to_tensor(target_price, dtype = tf.float32), (-1,1)))
    ypreds.append(ypred)
    
bundle_price = tf.concat(bundle_prices, 0)
if_buy_pop_gift = tf.concat(if_buy_pop_gifts, 0)
pred_bundle_price = tf.concat(ypreds, 0)

s = min(tf.shape(bundle_price)[0], tf.shape(pred_bundle_price)[0])

bundle_price = bundle_price[:s,]
if_buy_pop_gift = if_buy_pop_gift[:s,]
pred_bundle_price = pred_bundle_price[:s,]

badcase_flag = tf.logical_and( if_buy_pop_gift == 1 , tf.cast(bundle_price,tf.float32) > pred_bundle_price )

badcase_rate = tf.math.count_nonzero(badcase_flag).numpy() / tf.shape(badcase_flag)[0].numpy()
badcase_loss = tf.where(badcase_flag, tf.cast(bundle_price,tf.float32) - pred_bundle_price, tf.zeros_like(pred_bundle_price))
badcase_loss = tf.math.reduce_sum(badcase_loss).numpy() / 100
badcase_rate = '{:.2%}'.format(badcase_rate)

greycase_flag = tf.logical_and( if_buy_pop_gift == 0 , tf.cast(bundle_price,tf.float32) < pred_bundle_price )
greycase_rate = tf.math.count_nonzero(greycase_flag).numpy() / tf.shape(greycase_flag)[0].numpy()
greycase_rate = '{:.2%}'.format(greycase_rate)

print('Case closed')
 # 预测标签分布

y,idx,cnt = tf.unique_with_counts(tf.reshape(pred_bundle_price, (-1)))

# pred_price_distribution = dict(zip(y.numpy(),cnt.numpy()/cnt.numpy().sum()))
dist_normal = dict(zip(y.numpy(),cnt.numpy()/cnt.numpy().sum()))

badcase_rate, badcase_loss, greycase_rate

In [ ]:
# for f, conf in fe.feature_conf_map.items():
#     depend = ','.join(conf._depend)
#     s = f"f.append(FeatPlatConfig.FeatureConfig(feature_name = '{f}', depends = ['{depend}'], method = 'VectorFill', args = ['{conf._args[0]}','0']))"
#     # print(s)

In [ ]:
signature_dict = {
    k : v.type_spec
    for k,v in inputs.items()
    if k!='bundle_price' and k != 'trigger_type'    
}

@tf.function(input_signature=[signature_dict])
def serve(inputs):
    

    @tf.function
    def repeat(x, shape):
        x = tf.repeat(tf.expand_dims(x, 0), len(target_price), axis = 0)
        x = tf.reshape(x, (shape * len(target_price), -1))
        return x

    @tf.function
    def price(x, shape):
        x = tf.reshape(x, (shape, -1))
        x = tf.argmax(x, axis = 1)
        x = tf.gather(target_price_tensor_str, x)
        return x
    

    mask_trigger_type_tensor = tf.convert_to_tensor([['mask']])
    target_price = [499, 999, 1499, 1999, 2999, 4999, 9999]
    target_price_tensor = tf.convert_to_tensor(target_price)
    target_price_tensor_str =  tf.convert_to_tensor(["4.99", "9.99", "14.99", "19.99", "29.99", "49.99", "99.99"])
    
    x = {k:v for k,v in inputs.items()}

    shape = 1

    x['trigger_type'] = mask_trigger_type_tensor

    for k in x.keys():
        if k != 'bundle_price':
            x[k] = repeat(x[k], shape)
    x['bundle_price'] = tf.reshape(tf.repeat(target_price_tensor, shape), (-1,1))

    x = model(x)
    x = price(x, shape)
    return x

model.serve = serve
x, _ = next(iter(validdata.unbatch().batch(1)))
x.pop('bundle_price')
x.pop('trigger_type')
for key in x.keys():
    x[key] = tf.cast(x[key],tf.uint64)
model.serve(x)

In [ ]:
model.save(f'../../savedmodel/savedmodel/dynamic_pricing/0.0.1/1', signatures=model.serve)

In [ ]:
from tensorflow_serving.apis.predict_pb2 import PredictRequest
from tensorflow_serving.apis.prediction_service_pb2_grpc import *
from tensorflow_serving.apis.get_model_metadata_pb2 import *
from tensorflow_serving.apis.model_pb2 import *
from tensorflow_serving.apis.input_pb2 import *

stub = PredictionServiceStub(grpc.insecure_channel('localhost:8500'))
resp = stub.GetModelMetadata(GetModelMetadataRequest(model_spec = ModelSpec(name = 'dynamic_pricing_0.0.1'), metadata_field = ["signature_def"]))

s = SignatureDefMap()
resp.metadata['signature_def'].Unpack(s)
s.signature_def['serving_default'].inputs.__len__()

In [ ]:
from google.protobuf.any_pb2 import Any
a = Any(
    type_url="type.googleapis.com/tensorflow.serving.PredictRequest",
    value = b"\n\027\n\025dynamic_pricing_0.0.1\022 \n\006avg_pp\022\026\010\027\022\010\022\002\010\001\022\002\010\001\212\001\007\366\246\312\262\370\315\020\022 \n\006max_pp\022\026\010\027\022\010\022\002\010\001\022\002\010\001\212\001\007\202\327\320\277\263\342\010\022:\n\004gold\0222\010\027\022\010\022\002\010\001\022\002\010\005\212\001#\254\243\352\327\263\227\003\205\344\201\344\215\220\020\331\235\355\243\212\301\014\277\261\366\233\201\242)\274\331\366\361\230\235\"\022=\n\requipment_exp\022,\010\027\022\010\022\002\010\001\022\002\010\005\212\001\035\205\236\353\211\272\376\017\300\232\241\272\320\352\032\367\210\366\221\204\206\030\234\266\256\324\277\314/\000\022b\n\003p7d\022[\010\027\022\010\022\002\010\001\022\002\010\020\212\001L\352\245\305\267\221\210\036\250\246\372\201\250\2163\263\343\247\263\314\316\007\327\237\314\372\356\317\023\200\374\310\237\204\252\024\313\221\333\273\242\261\034\373\263\347\317\356\341$\313\321\241\253\322\256=\312\351\220\365\211\241\017\207\352\230\356\355\2575\000\000\000\000\000\000\022&\n\014dormancy_day\022\026\010\027\022\010\022\002\010\001\022\002\010\001\212\001\007\277\204\206\334\363\300%\022D\n\024common_equipment_exp\022,\010\027\022\010\022\002\010\001\022\002\010\005\212\001\035\205\236\353\211\272\376\017\331\235\355\243\212\301\014\367\210\366\221\204\206\030\213\240\326\215\306\261$\000\022\035\n\003sid\022\026\010\027\022\010\022\002\010\001\022\002\010\001\212\001\007\213\227\325\310\357\334=\022C\n\023arena_attack_ticket\022,\010\027\022\010\022\002\010\001\022\002\010\005\212\001\035\205\236\353\211\272\376\017\347\360\237\354\207\335\036\367\210\366\221\204\206\030\243\253\216\246\202\2253\000\022\036\n\004pcnt\022\026\010\027\022\010\022\002\010\001\022\002\010\001\212\001\007\354\374\267\376\330\344\024\022=\n\rcampaign_coin\022,\010\027\022\010\022\002\010\001\022\002\010\005\212\001\035\205\236\353\211\272\376\017\334\312\310\234\310\341\037\367\210\366\221\204\206\030\340\211\274\336\265\305\010\000\022!\n\007vip_lvl\022\026\010\027\022\010\022\002\010\001\022\002\010\001\212\001\007\230\351\220\200\363\203\030\022\"\n\010max_5rpp\022\026\010\027\022\010\022\002\010\001\022\002\010\001\212\001\007\241\361\257\263\253\216\033\022@\n\020group_army_point\022,\010\027\022\010\022\002\010\001\022\002\010\005\212\001\035\205\236\353\211\272\376\017\351\344\275\312\360\306\027\367\210\366\221\204\206\030\312\373\356\231\301\246\025\000\022c\n\004p30d\022[\010\027\022\010\022\002\010\001\022\002\010\020\212\001L\373\263\347\317\356\341$\313\321\241\253\322\256=\323\233\356\220\345\344\007\344\240\304\225\344\207%\263\343\247\263\314\316\007\327\237\314\372\356\317\023\200\374\310\237\204\252\024\313\221\333\273\242\261\034\236\317\311\264\270\337&\304\354\264\367\260\235\032\000\000\000\000\000\000\022\036\n\004lang\022\026\010\027\022\010\022\002\010\001\022\002\010\001\212\001\007\246\367\362\340\352\260\014\022@\n\narena_coin\0222\010\027\022\010\022\002\010\001\022\002\010\005\212\001#\254\243\352\327\263\227\003\271\374\361\261\377\3757\334\312\310\234\310\341\037\346\233\237\300\261\264>\231\354\216\211\335\217\014\022:\n\nguild_coin\022,\010\027\022\010\022\002\010\001\022\002\010\005\212\001\035\205\236\353\211\272\376\017\300\232\241\272\320\352\032\367\210\366\221\204\206\030\234\266\256\324\277\314/\000\022\035\n\003lvl\022\026\010\027\022\010\022\002\010\001\022\002\010\001\212\001\007\367\262\373\335\263\234(\022\037\n\005tot_p\022\026\010\027\022\010\022\002\010\001\022\002\010\001\212\001\007\361\223\205\347\323\337/\022=\n\007top_key\0222\010\027\022\010\022\002\010\001\022\002\010\005\212\001#\254\243\352\327\263\227\003\271\374\361\261\377\3757\273\306\271\226\271\233\032\276\321\223\346\207\2127\220\211\366\232\262\342=\022\036\n\004lped\022\026\010\027\022\010\022\002\010\001\022\002\010\001\212\001\007\377\324\230\232\224\2430\022$\n\nsvr_region\022\026\010\027\022\010\022\002\010\001\022\002\010\001\212\001\007\344\325\345\352\202\210\004\022\"\n\010lifespan\022\026\010\027\022\010\022\002\010\001\022\002\010\001\212\001\007\363\326\265\245\241\366\024\022\036\n\004type\022\026\010\027\022\010\022\002\010\001\022\002\010\001\212\001\007\317\200\301\374\262\307\032\022o\n\004p60d\022g\010\027\022\010\022\002\010\001\022\002\010\020\212\001X\263\343\247\263\314\316\007\327\237\314\372\356\317\023\373\263\347\317\356\341$\313\321\241\253\322\256=\200\374\310\237\204\252\024\313\221\333\273\242\261\034\323\233\356\220\345\344\007\344\240\304\225\344\207%\214\307\257\336\302\2679\201\347\354\267\375\245?\331\201\360\203\245\3560\304\354\264\367\260\235\032\000\000\000\000\022F\n\026airforce_equipment_exp\022,\010\027\022\010\022\002\010\001\022\002\010\005\212\001\035\205\236\353\211\272\376\017\303\235\311\354\254\3321\367\210\366\221\204\206\030\213\240\326\215\306\261$\000\022F\n\020aircampaign_coin\0222\010\027\022\010\022\002\010\001\022\002\010\005\212\001#\254\243\352\327\263\227\003\205\344\201\344\215\220\020\242\272\343\213\207\2335\213\347\200\327\246\235?\231\354\216\211\335\217\014\0223\n\003oil\022,\010\027\022\010\022\002\010\001\022\002\010\005\212\001\035\215\301\366\260\362\267\020\303\235\311\354\254\3321\277\261\366\233\201\242)\213\240\326\215\306\261$\000\0225\n\005metal\022,\010\027\022\010\022\002\010\001\022\002\010\005\212\001\035\276\204\316\212\346\2208\331\235\355\243\212\301\014\277\261\366\233\201\242)\213\214\334\210\203\326\023\000\022V\n\003p3d\022O\010\027\022\010\022\002\010\001\022\002\010\020\212\001@\373\263\347\317\356\341$\313\321\241\253\322\256=\200\374\310\237\204\252\024\313\221\333\273\242\261\034\263\343\247\263\314\316\007\327\237\314\372\356\317\023\352\245\305\267\221\210\036\250\246\372\201\250\2163\000\000\000\000\000\000\000\000\022 \n\006region\022\026\010\027\022\010\022\002\010\001\022\002\010\001\212\001\007\246\314\303\331\316\3777\022o\n\004p90d\022g\010\027\022\010\022\002\010\001\022\002\010\020\212\001X\263\343\247\263\314\316\007\327\237\314\372\356\317\023\200\374\310\237\204\252\024\313\221\333\273\242\261\034\331\201\360\203\245\3560\304\354\264\367\260\235\032\373\263\347\317\356\341$\313\321\241\253\322\256=\323\233\356\220\345\344\007\344\240\304\225\344\207%\214\307\257\336\302\2679\201\347\354\267\375\245?\000\000\000\000\022A\n\022officer_bar_ticket\022+\010\027\022\010\022\002\010\001\022\002\010\005\212\001\034\205\236\353\211\272\376\017\361\345\353\371\345F\367\210\366\221\204\206\030\243\253\216\246\202\2253\000\022\"\n\010avg_5rpp\022\026\010\027\022\010\022\002\010\001\022\002\010\001\212\001\007\346\352\320\345\347\215=\022=\n\010camp_key\0221\010\027\022\010\022\002\010\001\022\002\010\005\212\001\"\254\243\352\327\263\227\003\235\227\206\237\314\370\032\361\345\353\371\345F\276\321\223\346\207\2127\220\211\366\232\262\342="
)
p = PredictRequest()
a.Unpack(p)
stub.Predict(p)